In [1]:
import tensorflow as tf

In [2]:
import os

In [3]:
#os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/Mukesh-areo/CNN_claasaifier.mlflow' 
#os.environ['MLFLOW_TRACKING_USERNAME']='Mukesh-areo' 
#os.environ['MLFLOW_TRACKING_PASSWORD']='434cc55976d25211145784e69290bf348ce1e14a' 


In [4]:
os.chdir("../")

In [5]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class Evaluavationconfig:
    """
    Configuration for evaluating the performance of a model.
    """
    path_of_model: Path
    training_data: Path
    mlflow_uri: str
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [7]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories,save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> Evaluavationconfig:
        eval_config = Evaluavationconfig(
                path_of_model= "artifacts/training/model.h5",
                training_data= "artifacts/data_ingestion/Images",
                mlflow_uri="https://dagshub.com/Mukesh-areo/CNN_claasaifier.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
from urllib.parse import urlparse
import tensorflow as tf
from pathlib import Path
#import mlflow
#import mlflow.keras
class Evaluation:
    def __init__(self, config: Evaluavationconfig):
        self.config = config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model("artifacts/training/model.h5")
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

#    def log_into_mlflow(self):
#        mlflow.set_registry_uri(self.config.mlflow_uri)
#        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
#        with mlflow.start_run():
#            mlflow.log_params(self.config.all_params)
#            mlflow.log_metrics(

#                {"loss": self.score[0], "accuracy": self.score[1]}
#            )
#            if tracking_url_type_store != "file":

#                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16model")
#            else:
#                mlflow.keras.log_model(self.model,"model")

In [9]:
from distutils.command import config


try:
    config= ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
#    evaluation.log_into_mlflow()

except Exception as e:
    print(e)

Found 3 images belonging to 3 classes.
2/2 [==============================] - 3s 674ms/step - loss: 2140.5583 - accuracy: 0.3333
